Les exemples de ce TP sont visualisables sous forme de `Jupyter Notebooks`:

::: {.cell .markdown}

In [ ]:
#| echo: false
#| output: 'asis'
#| include: true
#| eval: true

import sys
sys.path.insert(1, '../../../../') #insert the utils module
from utils import print_badges

#print_badges(__file__)
print_badges("content/course/manipulation/02b_pandas_TP.qmd")

:::

Dans ce tutoriel `Pandas`, nous allons utiliser deux sources de données :

* Les émissions de gaz à effet de serre estimées au niveau communal par l'`ADEME`. Le jeu de données est 
disponible sur [data.gouv](https://www.data.gouv.fr/fr/datasets/inventaire-de-gaz-a-effet-de-serre-territorialise/#_)
et requêtable directement dans python avec
[cet url](https://koumoul.com/s/data-fair/api/v1/datasets/igt-pouvoir-de-rechauffement-global/convert).
`pandas` offre la possibilité d'importer des données directement depuis un url. C'est l'option
prise dans ce tutoriel.
Si vous préfèrez, pour des
raisons d'accès au réseau ou de performance, importer depuis un poste local,
vous pouvez télécharger les données et changer
les commandes d'import avec le chemin adéquat plutôt que l'url. 

* Idéalement, on utiliserait directement les données
[disponibles sur le site de l'Insee](https://www.insee.fr/fr/statistiques/3560121) mais celles-ci nécessitent un peu de travail
de nettoyage qui n'entre pas dans le cadre de ce TP. 
Pour faciliter l'import de données Insee, il est recommandé d'utiliser le package
[`pynsee`](https://github.com/InseeFrLab/Py-Insee-Data) qui simplifie l'accès aux données
de l'Insee disponibles sur le site web [insee.fr](https://www.insee.fr/fr/accueil)
ou via des API[^1]. 

[^1]: Toute contribution sur ce package, disponible sur [Github](https://github.com/InseeFrLab/Py-Insee-Data) est bienvenue !

Après avoir installé la librairie `pynsee` (voir l'[introduction à pandas](course/manipulation/02a_pandas_tutorial)), nous suivrons les conventions habituelles dans l'import des packages :


In [ ]:
#| echo: true

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pynsee
import pynsee.download

## Exploration de la structure des données

Commencer par importer les données de l'Ademe à l'aide du package `pandas`. Vous pouvez nommer le `DataFrame` obtenu `df`.


In [ ]:
#| echo: true

df = pd.read_csv("https://koumoul.com/s/data-fair/api/v1/datasets/igt-pouvoir-de-rechauffement-global/convert", sep=",")

Pour les données de cadrage au niveau communal (source Insee), le package `pynsee` facilite grandement la vie. 
La liste des données disponibles est [ici](https://inseefrlab.github.io/DoReMIFaSol/articles/donnees_dispo.html).
En l'occurrence, on va utiliser les données Filosofi (données de revenus) au niveau communal de 2016. 
Le point d'entrée principal de la fonction `pynsee` est la fonction `download_file`.
Le code pour télécharger les données est le suivant :


In [ ]:
#| echo: true

df_city = pynsee.download.download_file("FILOSOFI_COM_2016")

::: {.cell .markdown}

```{=html}
<div class="alert alert-info" role="alert">
<h3 class="alert-heading"><i class="fa-solid fa-comment"></i> Note</h3>
```


La fonction `download_file` attend un identifiant unique
pour savoir quelle base de données aller chercher et
restructurer depuis le
site [insee.fr](https://www.insee.fr/fr/accueil). 

Pour connaître la liste des bases disponibles, vous
pouvez utiliser la fonction `meta = pynsee.get_file_list()`. 
Celle-ci renvoie un `DataFrame` dans lequel on peut
rechercher, par exemple grâce à une recherche
de mot clé: 


In [ ]:
#| echo: true

meta = pynsee.get_file_list()
meta.loc[meta['label'].str.contains(r"Filosofi.*2016")]

Ici, `meta['label'].str.contains(r"Filosofi.*2016")` signifie:
"_`pandas` trouve moi tous les labels où sont contenus les termes Filosofi et 2016._"
 (`.*` signifiant "_peu m'importe le nombre de mots ou caractères entre_")



```{=html}
</div>
```

:::

::: {.cell .markdown}

```{=html}
<div class="alert alert-success" role="alert">
<h3 class="alert-heading"><i class="fa-solid fa-pencil"></i> Exercice 1: Afficher des données</h3>
```



L'objectif de cet exercice est de vous amener à afficher des informations sur les données dans un bloc de code (notebook) ou dans la console
    
Commencer sur `df`: 

1. Utiliser les méthodes adéquates pour les 10 premières valeurs, les 15 dernières et un échantillon aléatoire de 10 valeurs
2. Tirer 5 pourcent de l'échantillon sans remise
3. Ne conserver que les 10 premières lignes et tirer aléatoirement dans celles-ci pour obtenir un DataFrame de 100 données.
4. Faire 100 tirages à partir des 6 premières lignes avec une probabilité de 1/2 pour la première observation et une probabilité uniforme pour les autres
5. Faire la même chose sur `df_city`.


```{=html}
</div>
```

:::


In [ ]:
#| echo: false
# Question 1
df.head(10)
df.tail(15)
df.sample(10)

# Question 2
df.sample(frac = 0.05)

# Question 3
df[:10].sample(n = 100, replace = True)

# Question 4
df[:6].sample(n = 100, replace = True, weights = [0.5] + [0.1]*5)

# Question 5
df_city.head(10)
df_city.tail(15)
df_city.sample(10)
df_city.sample(frac = 0.05)
df_city[:10].sample(n = 100, replace = True)
df_city[:6].sample(n = 100, replace = True, weights = [0.5] + [0.1]*5)

Cette première approche exploratoire donne une idée assez précise de la manière dont les données sont organisées.
On remarque ainsi une différence entre `df` et `df_city` quant aux valeurs manquantes :
la première base est relativement complète, la seconde comporte beaucoup de valeurs manquantes.
Autrement dit, si on désire exploiter `df_city`, il faut faire attention à la variable choisie.


::: {.cell .markdown}

```{=html}
<div class="alert alert-success" role="alert">
<h3 class="alert-heading"><i class="fa-solid fa-pencil"></i> Exercice 2: structure des données</h3>
```



La première chose à vérifier est le format des données,
afin d'identifier des types de variables qui ne conviennent pas

Ici, comme c'est `pandas` qui a géré automatiquement les types de variables,
il y a peu de chances que les types ne soient pas adéquats mais une vérification ne fait pas de mal.

1. Vérifier les types des variables.
S'assurer que les types des variables communes aux deux bases sont cohérents.
Pour les variables qui ne sont pas en type `float` alors qu'elles devraient l'être, modifier leur type.

Ensuite, on vérifie les dimensions des `DataFrames` et la structure de certaines variables clés.
En l'occurrence, les variables fondamentales pour lier nos données sont les variables communales.
Ici, on a deux variables géographiques: un code commune et un nom de commune. 

2. Vérifier les dimensions des DataFrames

3. Vérifier le nombre de valeurs uniques des variables géographiques dans chaque base. Les résultats apparaissent-ils cohérents ?

4. Identifier dans `df_city` les noms de communes qui correspondent à plusieurs codes communes et sélectionner leurs codes. En d'autres termes, identifier les `CODGEO` tels qu'il existe des doublons de `LIBGEO` et les stocker dans un vecteur `x` (conseil: faire attention à l'index de `x`)

On se focalise temporairement sur les observations où le libellé comporte plus de deux codes communes différents

5. Regarder dans `df_city` ces observations

6. Pour mieux y voir, réordonner la base obtenue par order alphabétique

7. Déterminer la taille moyenne (variable nombre de personnes: `NBPERSMENFISC16`) et quelques statistiques descriptives de ces données.
Comparer aux mêmes statistiques sur les données où libellés et codes communes coïncident

8. Vérifier les grandes villes (plus de 100 000 personnes),
la proportion de villes pour lesquelles un même nom est associé à différents codes commune.

9. Vérifier dans `df_city` les villes dont le libellé est égal à Montreuil.
Vérifier également celles qui contiennent le terme 'Saint-Denis'


```{=html}
</div>
```

:::


In [ ]:
#| echo: false

# Question 1
print("base df")
print(df.dtypes)
print("\nbase df_city")
print(df_city.dtypes)

# Il faut changer les types de toutes les variables de df_city à l'exception des codes et libellés de commune.
df_city[df_city.columns[2:len(df_city.columns)]] = df_city[df_city.columns[2:len(df_city.columns)]].apply(pd.to_numeric)
print("\nbase df_city corrigée")
print(df_city.dtypes)

In [ ]:
#| echo: false

# Question 2
print(df.shape)
print(df_city.shape)

In [ ]:
#| echo: false

# Question 3
print(df[['INSEE commune', 'Commune']].nunique())
print(df_city[['CODGEO', 'LIBGEO']].nunique())
# Résultats dont l'ordre de grandeur est proche. Dans les deux cas, #(libelles) < #(code)

In [ ]:
#| echo: false

# Question 4
x = df_city.groupby('LIBGEO').count()['CODGEO']
x = x[x>1]
x = x.reset_index()
x

In [ ]:
#| echo: false
# Question 5
df_city[df_city['LIBGEO'].isin(x['LIBGEO'])]

In [ ]:
#| echo: false
# Question 6
df_city[df_city['LIBGEO'].isin(x['LIBGEO'])].sort_values('LIBGEO')

In [ ]:
#| echo: false
# Question 7
print(df_city[df_city['LIBGEO'].isin(x['LIBGEO'])]['NBPERSMENFISC16'].describe())
print(df_city[~df_city['LIBGEO'].isin(x['LIBGEO'])]['NBPERSMENFISC16'].describe())

In [ ]:
#| echo: false
# Question 8
df_big_city = df_city[df_city['NBPERSMENFISC16']>100000].copy()
df_big_city['probleme'] = df_big_city['LIBGEO'].isin(x['LIBGEO'])
df_big_city['probleme'].mean()
df_big_city[df_big_city['probleme']]

In [ ]:
#| echo: false
# Question 9
df_city[df_city.LIBGEO == 'Montreuil']
df_city[df_city.LIBGEO.str.contains('Saint-Denis')].head(10)

Ce petit exercice permet de se rassurer car les libellés dupliqués
sont en fait des noms de commune identiques mais qui ne sont pas dans le même département.
Il ne s'agit donc pas d'observations dupliquées.
On se fiera ainsi aux codes communes, qui eux sont uniques.

## Les indices

Les indices sont des éléments spéciaux d'un DataFrame puisqu'ils permettent d'identifier certaines observations.
Il est tout à fait possible d'utiliser plusieurs indices, par exemple si on a des niveaux imbriqués.


::: {.cell .markdown}

```{=html}
<div class="alert alert-success" role="alert">
<h3 class="alert-heading"><i class="fa-solid fa-pencil"></i> Exercice 3: Les indices</h3>
```



A partir de l'exercice précédent, on peut se fier aux codes communes.

1. Fixer comme indice la variable de code commune dans les deux bases.
Regarder le changement que cela induit sur le *display* du `DataFrame`

2. Les deux premiers chiffres des codes communes sont le numéro de département.
Créer une variable de département `dep` dans `df` et dans `df_city`

3. Calculer les émissions totales par secteur pour chaque département.
Mettre en log ces résultats dans un objet `df_log`.
Garder 5 départements et produire un `barplot`

4. Repartir de `df`.
Calculer les émissions totales par département et sortir la liste
des 10 principaux émetteurs de CO2 et des 5 départements les moins émetteurs.
Sans faire de *merge*,
regarder les caractéristiques de ces départements (population et niveau de vie)


```{=html}
</div>
```

:::


In [ ]:
#| echo: false
# Question 1
display(df)
df = df.set_index('INSEE commune')
display(df)

display(df_city)
df_city =  df_city.set_index('CODGEO') 
display(df_city)

In [ ]:
#| echo: false
# Question 2
df['dep'] = df.index.str[:2]
df_city['dep'] = df_city.index.str[:2]

In [ ]:
#| echo: false
# Question 3
df_log = df.groupby('dep').sum().apply(np.log)
print(df_log.head())
df_log.sample(5).plot(kind = "bar")

In [ ]:
#| echo: false
# Question 4
## Emissions totales par département (df)
df_emissions = df.reset_index().set_index(['INSEE commune','dep']).sum(axis = 1).groupby('dep').sum()
#df.reset_index().groupby('dep').sum().sum(axis = 1).head() #version simplifiee ?
gros_emetteurs = df_emissions.sort_values(ascending = False).head(10)
petits_emetteurs = df_emissions.sort_values().head(5)

## Caractéristiques des départements (df_city)
print("gros emetteurs")
print(df_city[df_city['dep'].isin(gros_emetteurs.index)][['NBPERSMENFISC16','MED16']].mean())
print("\npetits emetteurs")
print(df_city[df_city['dep'].isin(petits_emetteurs.index)][['NBPERSMENFISC16','MED16']].mean())
# Les petits emetteurs sont en moyenne plus pauvres et moins peuplés que les gros emetteurs

::: {.cell .markdown}

```{=html}
<div class="alert alert-success" role="alert">
<h3 class="alert-heading"><i class="fa-solid fa-pencil"></i> Exercice 4: performance des indices</h3>
```


Un des intérêts des indices est qu'ils permettent des agrégations efficaces. 

1. Repartir de `df` et créer une copie `df_copy = df.copy()` et `df_copy2 = df.copy()` afin de ne pas écraser le DataFrame `df`

2. Utiliser la variable `dep` comme indice pour `df_copy` et retirer tout index pour `df_copy2`

3. Importer le module `timeit` et comparer le temps d'exécution de la somme par secteur, pour chaque département, des émissions de CO2


```{=html}
</div>
```

:::


In [ ]:
#| echo: false
# Question 1
df_copy = df.copy()
df_copy2 = df.copy()

In [ ]:
#| echo: false
# Question 2
df_copy = df_copy.set_index('dep')
df_copy2 = df_copy2.reset_index()

In [ ]:
#| echo: false
#| eval: false
# Question 3
%timeit df_copy.drop('Commune', axis = 1).groupby('dep').sum()
%timeit df_copy2.drop('Commune', axis = 1).groupby('dep').sum()
# Le temps d'exécution est plus lent sur la base sans index par département.

# Restructurer les données

On présente généralement deux types de données : 
    
* format __wide__: les données comportent des observations répétées, pour un même individu (ou groupe), dans des colonnes différentes 
* format __long__: les données comportent des observations répétées, pour un même individu, dans des lignes différentes avec une colonne permettant de distinguer les niveaux d'observations

Un exemple de la distinction entre les deux peut être pris à l'ouvrage de référence d'Hadley Wickham, *R for Data Science*:

![](https://d33wubrfki0l68.cloudfront.net/3aea19108d39606bbe49981acda07696c0c7fcd8/2de65/images/tidy-9.png)


L'aide mémoire suivante aidera à se rappeler les fonctions à appliquer si besoin:

![](../02a_pandas_tutorial/reshape.png)

Le fait de passer d'un format *wide* au format *long* (ou vice-versa)
peut être extrêmement pratique car certaines fonctions sont plus adéquates sur une forme de données ou sur l'autre.
En règle générale, avec `Python` comme avec `R`, les formats *long* sont souvent préférables.

::: {.cell .markdown}

```{=html}
<div class="alert alert-success" role="alert">
<h3 class="alert-heading"><i class="fa-solid fa-pencil"></i> Exercice 5: Restructurer les données: wide to long</h3>
```


1. Créer une copie des données de l'`ADEME` en faisant `df_wide = df.copy()`

2. Restructurer les données au format *long* pour avoir des données d'émissions par secteur en gardant comme niveau d'analyse la commune (attention aux autres variables identifiantes).

3. Faire la somme par secteur et représenter graphiquement

4. Garder, pour chaque département, le secteur le plus polluant


```{=html}
</div>
```

:::


In [ ]:
#| echo: false
# Question 1

df_wide = df.copy()
df_wide[['Commune','dep', "Agriculture", "Tertiaire"]].head() 

In [ ]:
#| echo: false
# Question 2

df_wide.reset_index().melt(id_vars = ['INSEE commune','Commune','dep'],
                          var_name = "secteur", value_name = "emissions")

In [ ]:
#| echo: false
# Question 3

(df_wide.reset_index()
 .melt(id_vars = ['INSEE commune','Commune','dep'],
                          var_name = "secteur", value_name = "emissions")
 .groupby('secteur').sum().plot(kind = "barh")
)

In [ ]:
#| echo: false
# Question 4

(df_wide.reset_index().melt(id_vars = ['INSEE commune','Commune','dep'],
                          var_name = "secteur", value_name = "emissions")
 .groupby(['secteur','dep']).sum().reset_index().sort_values(['dep','emissions'], ascending = False).groupby('dep').head(1)
)

::: {.cell .markdown}

```{=html}
<div class="alert alert-success" role="alert">
<h3 class="alert-heading"><i class="fa-solid fa-pencil"></i> Exercice 6: long to wide</h3>
```


Cette transformation est moins fréquente car appliquer des fonctions par groupe, comme nous le verrons par la suite, est très simple. 

1. Repartir de `df_wide = df.copy()

2. Reconstruire le DataFrame, au format long, des données d'émissions par secteur en gardant comme niveau d'analyse la commune puis faire la somme par département et secteur

3. Passer au format *wide* pour avoir une ligne par secteur et une colonne par département

4. Calculer, pour chaque secteur, la place du département dans la hiérarchie des émissions nationales

5. A partir de là, en déduire le rang médian de chaque département dans la hiérarchie des émissions et regarder les 10 plus mauvais élèves, selon ce critère.


```{=html}
</div>
```

:::


In [ ]:
#| echo: false
# Question 1

df_wide = df.copy()

In [ ]:
#| echo: false
# Question 2

df_long_agg = (df_wide.reset_index()
 .melt(id_vars = ['INSEE commune','Commune','dep'],
                          var_name = "secteur", value_name = "emissions").groupby(["dep", "secteur"]).sum()
)

df_long_agg.head()

In [ ]:
#| echo: false
# Question 3

df_wide_agg = df_long_agg.reset_index().pivot_table(values = "emissions", index = "secteur", columns = "dep")

df_wide_agg.head()

In [ ]:
#| echo: false
# Question 4

df_wide_agg.rank(axis = 1)

In [ ]:
#| echo: false
# Question 5

df_wide_agg.rank(axis = 1).median().nlargest(10)

# Combiner les données

Une information que l'on cherche à obtenir s'obtient de moins en moins à partir d'une unique base de données. Il devient commun de devoir combiner des données issues de sources différentes. Nous allons ici nous focaliser sur le cas le plus favorable qui est la situation où une information permet d'apparier de manière exacte deux bases de données (autrement nous serions dans une situation, beaucoup plus complexe, d'appariement flou). La situation typique est l'appariement entre deux sources de données selon un identifiant individuel ou un identifiant de code commune, ce qui est notre cas.

Il est recommandé de lire [ce guide assez complet sur la question des jointures avec R](https://linogaliana.gitlab.io/documentationR/joindre-des-tables-de-donn%C3%A9es.html) qui donne des recommandations également utiles en `python`.

On utilise de manière indifférente les termes *merge* ou *join*. Le deuxième terme provient de la syntaxe SQL. En `pandas`, dans la plupart des cas, on peut utiliser indifféremment `df.join` et `df.merge`

![](../02a_pandas_tutorial/pandas_join.png)


::: {.cell .markdown}

```{=html}
<div class="alert alert-success" role="alert">
<h3 class="alert-heading"><i class="fa-solid fa-pencil"></i> Exercice 7: Calculer l'empreinte carbone par habitant</h3>
```


1. Créer une variable `emissions` qui correspond aux émissions totales d'une commune

2. Faire une jointure à gauche entre les données d'émissions et les données de cadrage. Comparer les émissions moyennes des villes sans *match* (celles dont des variables bien choisies de la table de droite sont NaN) avec celles où on a bien une valeur correspondante dans la base Insee

3. Faire un *inner join* puis calculer l'empreinte carbone (l'émission rapportée au nombre de ménages fiscaux) dans chaque commune. Sortir un histogramme en niveau puis en log et quelques statistiques descriptives sur le sujet. 

4. Regarder la corrélation entre les variables de cadrage et l'empreinte carbone. Certaines variables semblent-elles pouvoir potentiellement influer sur l'empreinte carbone ?


```{=html}
</div>
```

:::


In [ ]:
#| echo: false
# Question 1

df['emissions'] = df.sum(axis = 1)

In [ ]:
#| echo: false
# Question 2

df_merged = df.merge(df_city, how = "left", left_index = True, right_index = True)

print(df_merged[df_merged['LIBGEO'].isna()]['emissions'].mean())
print(df_merged[~df_merged['LIBGEO'].isna()]['emissions'].mean())

In [ ]:
#| echo: false
# Question 3

df_merged = df.merge(df_city, left_index = True, right_index = True)
df_merged['empreinte'] = df_merged['emissions']/df_merged['NBPERSMENFISC16']

df_merged['empreinte'].plot(kind ="hist")
np.log(df_merged['empreinte']).plot.hist()
df_merged['empreinte'].describe()

In [ ]:
#| echo: false
# Question 4

df_merged.corr()['empreinte'].nlargest(10)
# Les variables en lien avec le transport. 

# Exercices bonus

Les plus rapides d'entre vous sont invités à aller un peu plus loin en s'entraînant avec des exercices bonus qui proviennent du  [site de Xavier Dupré](http://www.xavierdupre.fr/app/ensae_teaching_cs/helpsphinx3). 3 notebooks en lien avec `numpy` et `pandas` vous y sont proposés : 

1. Calcul Matriciel, Optimisation : [énoncé](http://www.xavierdupre.fr/app/ensae_teaching_cs/helpsphinx3/notebooks/td2a_cenonce_session_2A.html) / [corrigé](http://www.xavierdupre.fr/app/ensae_teaching_cs/helpsphinx3/notebooks/td2a_correction_session_2A.html)
2. DataFrame et Graphes : [énoncé](http://www.xavierdupre.fr/app/ensae_teaching_cs/helpsphinx3/notebooks/td2a_cenonce_session_1.html) / [corrigé](http://www.xavierdupre.fr/app/ensae_teaching_cs/helpsphinx3/notebooks/td2a_correction_session_1.html)
3. Pandas et itérateurs : [énoncé](http://www.xavierdupre.fr/app/ensae_teaching_cs/helpsphinx3/notebooks/pandas_iterator.html) / [corrigé](http://www.xavierdupre.fr/app/ensae_teaching_cs/helpsphinx3/notebooks/pandas_iterator_correction.html)
